In [ ]:
import pandas as pd
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_rows",10)
pd.set_option("display.max_columns",5)
# DS= pd.read_csv('New_Few_IMDB_Dataset.csv');
df= pd.DataFrame(DS,columns =['review', 'sentiment','LLM_sentiment','LLM_confidence','LLM_FewShot']);
df


,review,sentiment,LLM_sentiment,LLM_confidence,LLM_FewShot
0,one of the other reviewers has mentioned that after watching just 1 oz episode youll be hooked t...,positive,negative,80,NaN
1,a wonderful little production br br the filming technique is very unassuming very oldtimebbc fas...,positive,positive,95,NaN
2,i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air ...,positive,positive,95,NaN
3,basically theres a family where a little boy jake thinks theres a zombie in his closet his pare...,negative,negative,70,negative
4,petter matteis love in the time of money is a visually stunning film to watch mr mattei offers u...,positive,positive,90,NaN
...,...,...,...,...,...
9995,fun entertaining movie about wwii german spy julie andrews falling in love with american pilot r...,positive,positive,80,NaN
9996,give me a break how can anyone say that this is a good hockey movie i know that movies tend to d...,negative,negative,90,NaN
9997,this movie is a bad movie but after watching an endless series of bad horror movies i can say th...,negative,negative,60,negative
9998,this is a movie that was probably made to entertain the middle school early high school age kids...,negative,negative,80,NaN


In [ ]:
def count_words(sentence):
    words = sentence.split()
    return len(words)

def average_word_count(dataset_column):
    total_word_count = 0
    num_sentences = len(dataset_column)

    for sentence in dataset_column:
        total_word_count += count_words(sentence)

    average = total_word_count / num_sentences
    return average
avg_word_count = average_word_count(df["review"])
print(f"Average word count: {avg_word_count:.2f}")

Average word count: 230.15


In [ ]:
X=df["review"]
Y=df["sentiment"]
Y_LLM=df["LLM_sentiment"]
LLM_confidence=df["LLM_confidence"]
LLM_FewShot=df["LLM_FewShot"]

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
k= 0.02;
test= 0.05;

In [ ]:
# mask = LLM_confidence > 70 #0.0703  Accuracy Increase = 0.9637    0.1388  True Label
# mask = LLM_confidence > 80 #0.0985  Accuracy Increase = 0.9919    0.4005  True Label
# mask = LLM_confidence > 90
# mask = LLM_confidence > 100
# Y_LLM_new = Y_LLM.where(mask, other=Y)
mask_few= LLM_confidence<=70
Y_LLM_new = LLM_FewShot.where(mask_few, other=Y_LLM)

In [ ]:
x=X;
y=Y;
# y_LLM=Y_LLM;
#apply mask
y_LLM=Y_LLM_new;
llm_confidence=LLM_confidence;
x_train=[None]*2;
y_train=[None]*2;
y_train_llm=[None]*2;
llm_confidence_train=[None]*2;

In [ ]:
Test_size=int(test*X.count());
# x, x_test, y, y_test, y_llm, y_llm_test,llm_confidence,llm_confidence_test = train_test_split(x, y, y_LLM,llm_confidence, test_size=Test_size, shuffle=False);
x, x_test, y, y_test, y_llm, y_llm_test,llm_confidence,llm_confidence_test = train_test_split(x, y, y_LLM,llm_confidence, test_size=Test_size,shuffle=True,random_state=100)

# x , x_test , y , y_test = train_test_split(x,y,test_size=Test_size,shuffle=True,random_state=100);
# x , x_test , y , y_test = train_test_split(x,y,test_size=Test_size,shuffle=True,random_state=120);
# x , x_test , y , y_test = train_test_split(x,y,test_size=Test_size,shuffle=True,random_state=130);
# x , x_test , y , y_test = train_test_split(x,y,test_size=Test_size,shuffle=True,random_state=140);

In [ ]:
train_size= int(k*X.count())
x_train[0] , x_train[1] , y_train[0] , y_train[1],y_train_llm[0],y_train_llm[1],llm_confidence_train[0],llm_confidence_train[1] = train_test_split(x,y, y_llm,llm_confidence,train_size=train_size,shuffle=False);

This part used for applying  model



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(penalty='l2', dual=False, tol=0.01, C=1, fit_intercept=True,
                        intercept_scaling=1, class_weight=None,random_state=None, solver='liblinear',
                        max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None);
vectorization = TfidfVectorizer();

In [ ]:
np.count_nonzero(Y_LLM==Y)/Y.size

0.8934

In [ ]:
temp=np.where(Y_LLM!=Y)
temp=temp[0]

In [ ]:
LLM_confidence[temp].size

1066

In [ ]:
print(np.count_nonzero(LLM_confidence[temp]>90)/Y.size)
print(np.count_nonzero(LLM_confidence>90)/Y.size)

0.0008
0.2185


In [ ]:
print(np.count_nonzero(LLM_confidence[temp]<=90)/Y.size," Accuracy Increase =" ,(np.count_nonzero(LLM_confidence[temp]<=90)+ np.count_nonzero(Y_LLM==Y))/Y.size)
print(np.count_nonzero(LLM_confidence<=90)/Y.size," True Label")

0.1058  Accuracy Increase = 0.9992
0.7815  True Label


In [ ]:
print(np.count_nonzero(LLM_confidence[temp]<=85)/Y.size," Accuracy Increase =",(np.count_nonzero(LLM_confidence[temp]<=85)+ np.count_nonzero(Y_LLM==Y))/Y.size)
print(np.count_nonzero(LLM_confidence<=85)/Y.size," True Label")

0.0985  Accuracy Increase = 0.9919
0.4005  True Label


In [ ]:
print(np.count_nonzero(LLM_confidence[temp]<=80)/Y.size," Accuracy Increase =",(np.count_nonzero(LLM_confidence[temp]<=80)+ np.count_nonzero(Y_LLM==Y))/Y.size)
print(np.count_nonzero(LLM_confidence<=80)/Y.size," True Label")

0.0985  Accuracy Increase = 0.9919
0.4005  True Label


In [ ]:
print(np.count_nonzero(LLM_confidence[temp]<=75)/Y.size," Accuracy Increase =",(np.count_nonzero(LLM_confidence[temp]<=75)+ np.count_nonzero(Y_LLM==Y))/Y.size)
print(np.count_nonzero(LLM_confidence<=75)/Y.size," True Label")

0.0703  Accuracy Increase = 0.9637
0.1388  True Label


In [ ]:
print(np.count_nonzero(LLM_confidence[temp]<=70)/Y.size," Accuracy Increase =",(np.count_nonzero(LLM_confidence[temp]<=70)+ np.count_nonzero(Y_LLM==Y))/Y.size)
print(np.count_nonzero(LLM_confidence <= 70)/Y.size," True Label")

0.0703  Accuracy Increase = 0.9637
0.1388  True Label


In [ ]:
print(np.count_nonzero(LLM_confidence[temp]<=65)/Y.size," Accuracy Increase =",(np.count_nonzero(LLM_confidence[temp]<=65)+ np.count_nonzero(Y_LLM==Y))/Y.size)
print(np.count_nonzero(LLM_confidence <= 65)/Y.size," True Label")

0.0085  Accuracy Increase = 0.9019
0.0112  True Label


In [ ]:
xv= vectorization.fit_transform(x_train[0]);
xv_test = vectorization.transform(x_test);
LR.fit(xv,y_train_llm[0]);
acc_Unchanged_Train_on_Test=LR.score(xv_test,y_test)
acc_Unchanged_Train_on_Test

0.632

In [ ]:
xv_test = vectorization.transform(x_train[1]);
acc_Unchanged_Train=LR.score(xv_test,y_train[1])
acc_Unchanged_Train

0.6449462365591397

med border illustrates the portion of data to add next iterate

In [ ]:
size_of_iteration =250;
acc_pool=[0]*(size_of_iteration+1);
acc_pool_known=[0]*(size_of_iteration+1);
acc_test_known=[0]*(size_of_iteration+1);

acc_Changed_Train_On_New_Test=[0]*(size_of_iteration+1);
acc_different_portion=[0]*(size_of_iteration+1);
med_border=0.002
Train_count=y_train[0].count();
med_fix_count=int(X.size*med_border)
acc_pool_tot=[0]*(size_of_iteration+1);
acc_test_tot=[0]*(size_of_iteration+1);
acc_different_portion_tot=[0]*(size_of_iteration+1);
Manual_count=[0]*(size_of_iteration+1);
Manual_count[0]=np.count_nonzero(llm_confidence_train[0]<=90);

In [ ]:
Manual_count[0]

158

In [ ]:
llm_confidence_train[0].size

200

In [ ]:
med_border*size_of_iteration+test

0.55

In [ ]:
k

0.02

The active learning part

In [ ]:
def Remove_data(True_vector, Ser):
    Adding = True_vector * Ser
    Adding.replace('', np.nan, inplace=True)
    Adding.dropna(inplace=True)
    return Adding

In [ ]:
def Remove_data_Num(True_vector, Ser):
    Adding = True_vector * Ser
    Adding.replace(0, np.nan, inplace=True)
    Adding.dropna(inplace=True)
    return Adding

In [ ]:
from sklearn.metrics import f1_score

Pool_x=x_train[1];
Pool_y=y_train[1];
Pool_y_LLM=y_train_llm[1];
Pool_LLM_Con=llm_confidence_train[1]

test_new_x=x_test;
test_new_y=y_test;
test_new_y_LLM=y_llm_test;
test_new_LLM_Con=llm_confidence_test;

removed_test_x= pd.Series();
removed_test_y= pd.Series();
removed_test_y_LLM= pd.Series();
removed_test_LLM_Con= pd.Series();

NewTrain_x=x_train[0];
NewTrain_y=y_train[0];
NewTrain_y_LLM=y_train_llm[0];
NewTrain_LLM_Con=llm_confidence_train[0];




for z in range(size_of_iteration+1):

  xv= vectorization.fit_transform(NewTrain_x);
  xv_test = vectorization.transform(Pool_x);
  LR.fit(xv,NewTrain_y_LLM);
  prediction= LR.predict(xv_test);
  prob=LR.predict_proba(xv_test)
  xv_test = vectorization.transform(test_new_x);
  prob2=LR.predict_proba(xv_test)

  # worst_prob = np.max(prob, axis=1)
  # threshold = np.quantile(worst_prob, med_fix_count / len(worst_prob))


  worst_prob = np.max(prob, axis=1)
  sorted_indices = np.argsort(worst_prob)
  threshold = worst_prob[sorted_indices[med_fix_count]]


  if z==0:
    threshold= 0


  # True_vector_pool=worst_prob < threshold

  True_vector_pool = np.zeros(Pool_x.size, dtype=bool)
  True_vector_pool[sorted_indices[:med_fix_count]] = worst_prob[sorted_indices[:med_fix_count]] <= threshold

  Adding_x=Remove_data(True_vector_pool,Pool_x)
  Adding_y=Remove_data(True_vector_pool,Pool_y)
  Adding_y_llm=Remove_data(True_vector_pool,Pool_y_LLM)
  Adding_llm_con=Remove_data_Num(True_vector_pool,Pool_LLM_Con)
  if z!=0:
    Manual_count[z]=Manual_count[z-1]+np.count_nonzero(Adding_llm_con<=70)

  NewTrain_x=pd.concat([NewTrain_x,Adding_x], ignore_index=True);
  NewTrain_y=pd.concat([NewTrain_y,Adding_y], ignore_index=True);
  NewTrain_y_LLM=pd.concat([NewTrain_y_LLM,Adding_y_llm], ignore_index=True);
  NewTrain_LLM_Con=pd.concat([NewTrain_LLM_Con,Adding_llm_con], ignore_index=True);

  worst_prob2 = np.max(prob2, axis=1)
  True_vector_test=worst_prob2 <= threshold
  Adding_x_test=Remove_data(True_vector_test,test_new_x)
  Adding_y_test=Remove_data(True_vector_test,test_new_y)
  Adding_y_test_llm=Remove_data(True_vector_test,test_new_y_LLM)
  Adding_test_llm_con=Remove_data_Num(True_vector_test,test_new_LLM_Con)

  removed_test_x=pd.concat([removed_test_x,Adding_x_test], ignore_index=True);
  removed_test_y=pd.concat([removed_test_y,Adding_y_test], ignore_index=True);
  removed_test_y_LLM=pd.concat([removed_test_y_LLM,Adding_y_test_llm], ignore_index=True);
  removed_test_LLM_Con=pd.concat([removed_test_LLM_Con,Adding_test_llm_con], ignore_index=True);

  True_vector_test_removing=np.logical_not(True_vector_test)
  test_new_x=Remove_data(True_vector_test_removing,test_new_x)
  test_new_y=Remove_data(True_vector_test_removing,test_new_y)
  test_new_y_LLM=Remove_data(True_vector_test_removing,test_new_y_LLM)
  test_new_LLM_Con=Remove_data_Num(True_vector_test_removing,test_new_LLM_Con)


  True_vector_pool_remained=np.logical_not(True_vector_pool)
  Pool_x=Remove_data(True_vector_pool_remained,Pool_x)
  Pool_y=Remove_data(True_vector_pool_remained,Pool_y)
  Pool_y_LLM=Remove_data(True_vector_pool_remained,Pool_y_LLM)
  Pool_LLM_Con=Remove_data_Num(True_vector_pool_remained,Pool_LLM_Con)


  print(NewTrain_x.size)
  print(Pool_x.size)
  print(test_new_x.size)

  xv= vectorization.fit_transform(NewTrain_x);
  LR.fit(xv,NewTrain_y_LLM)

  xv_test = vectorization.transform(Pool_x);
  acc_pool[z]=f1_score(Pool_y,LR.predict(xv_test), average='weighted');
  acc_pool_known[z]=f1_score(Pool_y_LLM,LR.predict(xv_test), average='weighted');
  tt=np.count_nonzero(NewTrain_y==NewTrain_y_LLM)/NewTrain_y.size
  # acc_pool_tot[z]=(tt*NewTrain_y.size+acc_pool[z]*Pool_y.size)/(NewTrain_y.size+Pool_y.size)


  xv_test2 = vectorization.transform(NewTrain_x);
  acc_Changed_Train_On_Train=f1_score(NewTrain_y,LR.predict(xv_test2), average='weighted');


  xv_test4 = vectorization.transform(test_new_x);
  acc_Changed_Train_On_New_Test[z]=f1_score(test_new_y,LR.predict(xv_test4), average='weighted');
  acc_test_known[z]=f1_score(test_new_y_LLM,LR.predict(xv_test4), average='weighted');
  if(removed_test_y.size>0):
    tt2=np.count_nonzero(removed_test_y==removed_test_y_LLM)/removed_test_y.size
  else:
    tt2=0;
  # acc_test_tot[z]=(tt2*removed_test_y.size+acc_Changed_Train_On_New_Test[z]*test_new_y.size)/(removed_test_y.size+test_new_y.size)

  Train_portion=int((k+(med_border)*(z))*X.count())
  print(Train_portion)

  x_temp, x_test_temp, y_temp, y_test_temp, y_llm_temp, y_llm_test_temp,llm_confidence_temp,llm_confidence_test_temp = train_test_split(x, y, y_llm,llm_confidence, train_size=Train_portion,shuffle=True,random_state=100)


  xv= vectorization.fit_transform(x_temp);
  LR.fit(xv,y_llm_temp);
  xv_test5 = vectorization.transform(x_test_temp);
  acc_different_portion[z]=f1_score(y_test_temp,LR.predict(xv_test5), average='weighted');
  tt3=np.count_nonzero(y_temp==y_llm_temp)/y_temp.size
  # acc_different_portion_tot[z]=(tt3*y_temp.size+acc_different_portion[z]*y_test_temp.size)/(y_temp.size+y_test_temp.size)





  print(acc_pool)
  # print(acc_pool_tot)

  print(acc_different_portion)
  # print(acc_different_portion_tot)

  print(acc_Changed_Train_On_New_Test)
  # print(acc_test_tot)

  print(acc_test_known)
  print(acc_pool_known)
  print('count: ', Manual_count)



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# import pandas as pd

# save=pd.DataFrame(acc_pool)
# save.to_excel(f"acc_pool_few.xlsx")
# save=pd.DataFrame(acc_pool_tot)
# save.to_excel(f"acc_pool_tot_few.xlsx")
# save=pd.DataFrame(acc_different_portion)
# save.to_excel(f"acc_different_portion_few.xlsx")
# save=pd.DataFrame(acc_different_portion_tot)
# save.to_excel(f"acc_different_portion_tot_few.xlsx")
# save=pd.DataFrame(acc_Changed_Train_On_New_Test)
# save.to_excel(f"acc_Changed_Train_On_New_Test_few.xlsx")
# save=pd.DataFrame(acc_test_tot)
# save.to_excel(f"acc_test_tot_few.xlsx")
# save=pd.DataFrame(acc_test_known)
# save.to_excel(f"acc_test_known_few.xlsx")
# save=pd.DataFrame(acc_pool_known)
# save.to_excel(f"acc_pool_known_few.xlsx")
# save=pd.DataFrame(Manual_count)
# save.to_excel(f"Manual_count_few.xlsx")
